###### > # **Part - I**

*   define actor critic networks
*   define PPO algorithm

################################################################################

In [13]:


############################### Import libraries ###############################


import os
import glob
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical

import numpy as np

import gym
import pongGym

################################## set device ##################################

print("============================================================================================")


# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
    
print("============================================================================================")




################################## PPO Policy ##################################


class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, action_std_init):
        super(ActorCritic, self).__init__()

#         self.quant = torch.quantization.QuantStub()
        self.actor = nn.Sequential(
                        torch.quantization.QuantStub(),
                        nn.Linear(state_dim, 4),
                        nn.ReLU(),
                        nn.Linear(4, 4),
                        nn.ReLU(),
                        nn.Linear(4, 4), 
                        nn.ReLU(),
                        nn.Linear(4, 4), 
                        nn.ReLU(),
                        nn.Linear(4, action_dim),
                        torch.quantization.DeQuantStub(),
                        nn.Softmax(dim=-1)
                    )
#         self.dequant = torch.quantization.DeQuantStub()
        # critic
        self.critic = nn.Sequential(
                        nn.Linear(state_dim, 64),
                        nn.ReLU(),
                        nn.Linear(64, 64),
                        nn.ReLU(),
                        nn.Linear(64, 64),
                        nn.ReLU(),
                        nn.Linear(64, 64), 
                        nn.ReLU(),
                        nn.Linear(64, 1)
                    )
    def forward(self):
        raise NotImplementedError
    

    def act(self, state):

        
        action_probs = self.actor(state)
        dist = Categorical(action_probs)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        
        return action.detach(), action_logprob.detach()
    

    def evaluate(self, state, action):


        action_probs = self.actor(state)
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state)
        
        return action_logprobs, state_values, dist_entropy


class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std_init=0.6):

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()


    def select_action(self, state):

        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            action, action_logprob = self.policy_old.act(state)

        self.buffer.states.append(state)
        self.buffer.actions.append(action)
        self.buffer.logprobs.append(action_logprob)

        return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)

        
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss
            advantages = rewards - state_values.detach()   
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        
        
       



Device set to : NVIDIA GeForce RTX 3070


################################################################################
> # **Part - II**

*   train PPO algorithm on environments
*   save preTrained networks weights and log files

################################################################################

In [ ]:

print("============================================================================================")


################################### Training ###################################


####### initialize environment hyperparameters ######

env_name = "pong_game"

max_ep_len = 60*120                    # max timesteps in one episode
max_training_timesteps = int(1e5)   # break training loop if timeteps > max_training_timesteps

print_freq = max_ep_len * 3     # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len * 2       # log avg reward in the interval (in num timesteps)
save_model_freq = int(2e4)      # save model frequency (in num timesteps)

action_std = None


#####################################################


## Note : print/log frequencies should be > than max_ep_len


################ PPO hyperparameters ################


update_timestep = max_ep_len * 10      # update policy every n timesteps
K_epochs = 60*10               # update policy for K epochs
# K_epochs = 4
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0001       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 910426         # set random seed if required (0 = no random seed)

#####################################################


print("training environment name : " + env_name)

env = pongGym.DoublePong()

# state space dimension
state_dim = env.observation_space.shape[0]

# action space dimension

action_dim = env.action_space.n



###################### logging ######################

#### log files for multiple runs are NOT overwritten

log_dir = "PPO_logs"
if not os.path.exists(log_dir):
      os.makedirs(log_dir)

log_dir = log_dir + '/' + env_name + '/'
if not os.path.exists(log_dir):
      os.makedirs(log_dir)


#### get number of log files in log directory
run_num = 0
current_num_files = next(os.walk(log_dir))[2]
run_num = len(current_num_files)


#### create new log file for each run 
log_f_name = log_dir + '/PPO_' + env_name + "_log_" + str(run_num) + ".csv"

print("current logging run number for " + env_name + " : ", run_num)
print("logging at : " + log_f_name)

#####################################################


################### checkpointing ###################

run_num_pretrained = 2      #### change this to prevent overwriting weights in same env_name folder

directory = "PPO_preTrained"
if not os.path.exists(directory):
      os.makedirs(directory)

directory = directory + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)

#####################################################


############# print all hyperparameters #############

print("--------------------------------------------------------------------------------------------")

print("max training timesteps : ", max_training_timesteps)
print("max timesteps per episode : ", max_ep_len)

print("model saving frequency : " + str(save_model_freq) + " timesteps")
print("log frequency : " + str(log_freq) + " timesteps")
print("printing average reward over episodes in last : " + str(print_freq) + " timesteps")

print("--------------------------------------------------------------------------------------------")

print("state space dimension : ", state_dim)
print("action space dimension : ", action_dim)

print("--------------------------------------------------------------------------------------------")

print("Initializing a discrete action space policy")

print("--------------------------------------------------------------------------------------------")

print("PPO update frequency : " + str(update_timestep) + " timesteps") 
print("PPO K epochs : ", K_epochs)
print("PPO epsilon clip : ", eps_clip)
print("discount factor (gamma) : ", gamma)

print("--------------------------------------------------------------------------------------------")

print("optimizer learning rate actor : ", lr_actor)
print("optimizer learning rate critic : ", lr_critic)

if random_seed:
    print("--------------------------------------------------------------------------------------------")
    print("setting random seed to ", random_seed)
    torch.manual_seed(random_seed)
#     env.seed(random_seed)
    np.random.seed(random_seed)

#####################################################

print("============================================================================================")

################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std)


# track total training time
start_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)

print("============================================================================================")


# logging file
log_f = open(log_f_name,"w+")
log_f.write('episode,timestep,reward\n')


# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

log_running_reward = 0
log_running_episodes = 0

time_step = 0
i_episode = 0

Max_reward = 0
Max_episode = 0
# training loop
# env = pongGym.DoublePong()
while i_episode <= 100000:
    
    state = env.reset()
    current_ep_reward = 0
    for t in range(1, max_ep_len+1):
        
        # select action with policy
        action = ppo_agent.select_action(state)
        state, reward, done, _ = env.step(action)
#         if i_episode > 100:
#             env.render()
        
        # saving reward and is_terminals
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)
        
        time_step +=1
        current_ep_reward += reward

        # update PPO agent
        if time_step % update_timestep == 0:
            ppo_agent.update()

        # log in logging file
        if time_step % log_freq == 0:

            # log average reward till last episode
            log_avg_reward = log_running_reward / log_running_episodes
            log_avg_reward = round(log_avg_reward, 4)

            log_f.write('{},{},{}\n'.format(i_episode, time_step, log_avg_reward))
            log_f.flush()

            log_running_reward = 0
            log_running_episodes = 0
            
        # printing average reward
        if time_step % print_freq == 0:

            # print average reward till last episode
            print_avg_reward = print_running_reward / print_running_episodes
            print_avg_reward = round(print_avg_reward, 2)

            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))
            print_running_reward = 0
            print_running_episodes = 0
        # break; if the episode is over
        if done:
            break

    # save model weights
#     print("Episode : {} \t\t Totol reward : {}".format(i_episode, current_ep_reward))
    if i_episode >= Max_episode + 1000:
        Max_episode = i_episode
        checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, int(Max_episode), run_num_pretrained)
        print("--------------------------------------------------------------------------------------------")
        print("saving model at : " + checkpoint_path)
        ppo_agent.save(checkpoint_path)
        print("model saved")
        print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
        print("--------------------------------------------------------------------------------------------") 
    print_running_reward += current_ep_reward
    print_running_episodes += 1

    log_running_reward += current_ep_reward
    log_running_episodes += 1

    i_episode += 1


log_f.close()
env.close()




# print total training time
print("============================================================================================")
end_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)
print("Finished training at (GMT) : ", end_time)
print("Total training time  : ", end_time - start_time)
print("============================================================================================")







training environment name : pong_game
current logging run number for pong_game :  10
logging at : PPO_logs/pong_game//PPO_pong_game_log_10.csv
--------------------------------------------------------------------------------------------
max training timesteps :  100000
max timesteps per episode :  7200
model saving frequency : 20000 timesteps
log frequency : 14400 timesteps
printing average reward over episodes in last : 21600 timesteps
--------------------------------------------------------------------------------------------
state space dimension :  24
action space dimension :  4
--------------------------------------------------------------------------------------------
Initializing a discrete action space policy
--------------------------------------------------------------------------------------------
PPO update frequency : 72000 timesteps
PPO K epochs :  600
PPO epsilon clip :  0.2
discount factor (gamma) :  0.99
------------------------------------------------------------------

Episode : 326 		 Timestep : 2354400 		 Average Reward : 156.59
Episode : 329 		 Timestep : 2376000 		 Average Reward : 226.76
Episode : 332 		 Timestep : 2397600 		 Average Reward : 207.02
Episode : 335 		 Timestep : 2419200 		 Average Reward : 236.03
Episode : 338 		 Timestep : 2440800 		 Average Reward : 338.79
Episode : 341 		 Timestep : 2462400 		 Average Reward : 308.86
Episode : 344 		 Timestep : 2484000 		 Average Reward : 292.02
Episode : 347 		 Timestep : 2505600 		 Average Reward : 274.87
Episode : 350 		 Timestep : 2527200 		 Average Reward : 385.19
Episode : 353 		 Timestep : 2548800 		 Average Reward : 360.36
Episode : 356 		 Timestep : 2570400 		 Average Reward : 381.14
Episode : 359 		 Timestep : 2592000 		 Average Reward : 313.58
Episode : 362 		 Timestep : 2613600 		 Average Reward : 414.1
Episode : 365 		 Timestep : 2635200 		 Average Reward : 442.53
Episode : 368 		 Timestep : 2656800 		 Average Reward : 349.63
Episode : 371 		 Timestep : 2678400 		 Average Reward : 

Episode : 719 		 Timestep : 5184000 		 Average Reward : 431.87
Episode : 722 		 Timestep : 5205600 		 Average Reward : 449.09
Episode : 725 		 Timestep : 5227200 		 Average Reward : 391.43
Episode : 728 		 Timestep : 5248800 		 Average Reward : 406.01
Episode : 731 		 Timestep : 5270400 		 Average Reward : 347.37
Episode : 734 		 Timestep : 5292000 		 Average Reward : 508.39
Episode : 737 		 Timestep : 5313600 		 Average Reward : 446.24
Episode : 740 		 Timestep : 5335200 		 Average Reward : 406.76
Episode : 743 		 Timestep : 5356800 		 Average Reward : 407.76
Episode : 746 		 Timestep : 5378400 		 Average Reward : 475.96
Episode : 749 		 Timestep : 5400000 		 Average Reward : 397.49
Episode : 752 		 Timestep : 5421600 		 Average Reward : 423.94
Episode : 755 		 Timestep : 5443200 		 Average Reward : 327.6
Episode : 758 		 Timestep : 5464800 		 Average Reward : 309.35
Episode : 761 		 Timestep : 5486400 		 Average Reward : 338.48
Episode : 764 		 Timestep : 5508000 		 Average Reward : 

Episode : 1097 		 Timestep : 7905600 		 Average Reward : 435.83
Episode : 1100 		 Timestep : 7927200 		 Average Reward : 488.32
Episode : 1103 		 Timestep : 7948800 		 Average Reward : 468.63
Episode : 1106 		 Timestep : 7970400 		 Average Reward : 469.29
Episode : 1109 		 Timestep : 7992000 		 Average Reward : 414.34
Episode : 1112 		 Timestep : 8013600 		 Average Reward : 517.16
Episode : 1115 		 Timestep : 8035200 		 Average Reward : 452.92
Episode : 1118 		 Timestep : 8056800 		 Average Reward : 430.66
Episode : 1121 		 Timestep : 8078400 		 Average Reward : 442.3
Episode : 1124 		 Timestep : 8100000 		 Average Reward : 510.79
Episode : 1127 		 Timestep : 8121600 		 Average Reward : 458.32
Episode : 1130 		 Timestep : 8143200 		 Average Reward : 516.07
Episode : 1133 		 Timestep : 8164800 		 Average Reward : 396.58
Episode : 1136 		 Timestep : 8186400 		 Average Reward : 298.73
Episode : 1139 		 Timestep : 8208000 		 Average Reward : 461.1
Episode : 1142 		 Timestep : 8229600 		 Av

Episode : 1481 		 Timestep : 10670400 		 Average Reward : 512.71
Episode : 1484 		 Timestep : 10692000 		 Average Reward : 555.33
Episode : 1487 		 Timestep : 10713600 		 Average Reward : 484.53
Episode : 1490 		 Timestep : 10735200 		 Average Reward : 515.07
Episode : 1493 		 Timestep : 10756800 		 Average Reward : 570.71
Episode : 1496 		 Timestep : 10778400 		 Average Reward : 564.69
Episode : 1499 		 Timestep : 10800000 		 Average Reward : 488.17
Episode : 1502 		 Timestep : 10821600 		 Average Reward : 504.51
Episode : 1505 		 Timestep : 10843200 		 Average Reward : 468.33
Episode : 1508 		 Timestep : 10864800 		 Average Reward : 458.28
Episode : 1511 		 Timestep : 10886400 		 Average Reward : 453.37
Episode : 1514 		 Timestep : 10908000 		 Average Reward : 401.75
Episode : 1517 		 Timestep : 10929600 		 Average Reward : 461.93
Episode : 1520 		 Timestep : 10951200 		 Average Reward : 453.48
Episode : 1523 		 Timestep : 10972800 		 Average Reward : 458.95
Episode : 1526 		 Timeste

...

Episode : 54620 		 Timestep : 393271200 		 Average Reward : 579.81
Episode : 54623 		 Timestep : 393292800 		 Average Reward : 446.12
Episode : 54626 		 Timestep : 393314400 		 Average Reward : 564.72
Episode : 54629 		 Timestep : 393336000 		 Average Reward : 542.82
Episode : 54632 		 Timestep : 393357600 		 Average Reward : 525.52
Episode : 54635 		 Timestep : 393379200 		 Average Reward : 534.52
Episode : 54638 		 Timestep : 393400800 		 Average Reward : 431.35
Episode : 54641 		 Timestep : 393422400 		 Average Reward : 506.1
Episode : 54644 		 Timestep : 393444000 		 Average Reward : 351.5
Episode : 54647 		 Timestep : 393465600 		 Average Reward : 373.25
Episode : 54650 		 Timestep : 393487200 		 Average Reward : 391.85
Episode : 54653 		 Timestep : 393508800 		 Average Reward : 484.16
Episode : 54656 		 Timestep : 393530400 		 Average Reward : 503.36
Episode : 54659 		 Timestep : 393552000 		 Average Reward : 496.77
Episode : 54662 		 Timestep : 393573600 		 Average Reward : 508.

Episode : 54989 		 Timestep : 395928000 		 Average Reward : 569.15
Episode : 54992 		 Timestep : 395949600 		 Average Reward : 522.92
Episode : 54995 		 Timestep : 395971200 		 Average Reward : 549.99
Episode : 54998 		 Timestep : 395992800 		 Average Reward : 581.71
--------------------------------------------------------------------------------------------
saving model at : PPO_preTrained/pong_game/PPO_pong_game_55000_2.pth
model saved
Elapsed Time  :  1 day, 21:55:45
--------------------------------------------------------------------------------------------
Episode : 55001 		 Timestep : 396014400 		 Average Reward : 523.46
Episode : 55004 		 Timestep : 396036000 		 Average Reward : 568.29
Episode : 55007 		 Timestep : 396057600 		 Average Reward : 524.04
Episode : 55010 		 Timestep : 396079200 		 Average Reward : 539.83
Episode : 55013 		 Timestep : 396100800 		 Average Reward : 495.11
Episode : 55016 		 Timestep : 396122400 		 Average Reward : 515.53
Episode : 55019 		 Timestep : 

Episode : 55343 		 Timestep : 398476800 		 Average Reward : 473.0
Episode : 55346 		 Timestep : 398498400 		 Average Reward : 493.38
Episode : 55349 		 Timestep : 398520000 		 Average Reward : 581.55
Episode : 55352 		 Timestep : 398541600 		 Average Reward : 523.56
Episode : 55355 		 Timestep : 398563200 		 Average Reward : 525.23
Episode : 55358 		 Timestep : 398584800 		 Average Reward : 512.71
Episode : 55361 		 Timestep : 398606400 		 Average Reward : 535.93
Episode : 55364 		 Timestep : 398628000 		 Average Reward : 422.04
Episode : 55367 		 Timestep : 398649600 		 Average Reward : 502.53
Episode : 55370 		 Timestep : 398671200 		 Average Reward : 557.49
Episode : 55373 		 Timestep : 398692800 		 Average Reward : 569.86
Episode : 55376 		 Timestep : 398714400 		 Average Reward : 511.51
Episode : 55379 		 Timestep : 398736000 		 Average Reward : 510.59
Episode : 55382 		 Timestep : 398757600 		 Average Reward : 534.7
Episode : 55385 		 Timestep : 398779200 		 Average Reward : 559.

Episode : 55712 		 Timestep : 401133600 		 Average Reward : 537.6
Episode : 55715 		 Timestep : 401155200 		 Average Reward : 453.44
Episode : 55718 		 Timestep : 401176800 		 Average Reward : 487.18
Episode : 55721 		 Timestep : 401198400 		 Average Reward : 513.36
Episode : 55724 		 Timestep : 401220000 		 Average Reward : 438.21
Episode : 55727 		 Timestep : 401241600 		 Average Reward : 451.81
Episode : 55730 		 Timestep : 401263200 		 Average Reward : 589.45
Episode : 55733 		 Timestep : 401284800 		 Average Reward : 505.41
Episode : 55736 		 Timestep : 401306400 		 Average Reward : 509.26
Episode : 55739 		 Timestep : 401328000 		 Average Reward : 472.6
Episode : 55742 		 Timestep : 401349600 		 Average Reward : 484.56
Episode : 55745 		 Timestep : 401371200 		 Average Reward : 545.3
Episode : 55748 		 Timestep : 401392800 		 Average Reward : 473.85
Episode : 55751 		 Timestep : 401414400 		 Average Reward : 472.87
Episode : 55754 		 Timestep : 401436000 		 Average Reward : 489.3

Episode : 56069 		 Timestep : 403704000 		 Average Reward : 519.4
Episode : 56072 		 Timestep : 403725600 		 Average Reward : 569.97
Episode : 56075 		 Timestep : 403747200 		 Average Reward : 513.53
Episode : 56078 		 Timestep : 403768800 		 Average Reward : 507.69
Episode : 56081 		 Timestep : 403790400 		 Average Reward : 523.07
Episode : 56084 		 Timestep : 403812000 		 Average Reward : 520.89
Episode : 56087 		 Timestep : 403833600 		 Average Reward : 553.48
Episode : 56090 		 Timestep : 403855200 		 Average Reward : 506.81
Episode : 56093 		 Timestep : 403876800 		 Average Reward : 490.35
Episode : 56096 		 Timestep : 403898400 		 Average Reward : 509.49
Episode : 56099 		 Timestep : 403920000 		 Average Reward : 541.09
Episode : 56102 		 Timestep : 403941600 		 Average Reward : 483.74
Episode : 56105 		 Timestep : 403963200 		 Average Reward : 542.76
Episode : 56108 		 Timestep : 403984800 		 Average Reward : 575.45
Episode : 56111 		 Timestep : 404006400 		 Average Reward : 521

In [ ]:
checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, "final", run_num_pretrained)
print("--------------------------------------------------------------------------------------------")
print("saving model at : " + checkpoint_path)
ppo_agent.save(checkpoint_path)
print("model saved")
print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
print("--------------------------------------------------------------------------------------------")

In [ ]:


# checkpoint_path = "PPO_pong_game_16000_1.pth"
# checkpoint_path = "./PPO_preTrained/pong_game/PPO_pong_game_1798_0.pth"
# ppo_agent.load(checkpoint_path)
env = pongGym.DoublePong()
state = env.reset()
for t in range(1, max_ep_len+1):

    # select action with policy
    action = ppo_agent.select_action(state)
    state, reward, done, _ = env.step(action)
#         if i_episode > 100:
    env.render()
    time.sleep(1/180)


    if done: 
        break

env.close()


